In [0]:

%run "./silvercommonutilities"

In [0]:
dbutils.widgets.text('modulename','sales')

In [0]:
modulename = dbutils.widgets.get('modulename')
config = load_config(f"/Volumes/bronze/da_sales_dev/salesvolume/silverconfig{modulename}.json")

for tables in config['tables']:
   try:
            table = tables['name']
            df = spark.read.table(f"bronze.da_{modulename}_dev.{table}")
            if tables.get('selectschema'):
               df = selectschema(df,tables['selectschema'])
            if tables.get('dropcolumns'):
               df = dropcolumns(df,tables['dropcolumns'])
            if tables.get('renamecolumns'):
               df = renamecolumns(df,tables['renamecolumns'])
            if True:
               df = dropduplicates(df)
            if tables.get('notnullwithor'):
               df = notnullwithor(df,tables['notnullwithor'])
            if tables.get('notnullwithand'):
               df = notnullwithand(df,tables['notnullwithand'])
            if tables.get('picklatestrecords'):
               df = picklatestrecords(df,tables['picklatestrecords'][0],tables['picklatestrecords'][1])
            if tables.get('schemaenforcemnet'):
               df = schemaenforcemnet(df,tables['schemaenforcemnet'])
            if tables.get('makenullstoemptystrings'):
               df = makenullstoemptystrings(df,tables['makenullstoemptystrings'])
            if tables.get('concatcolumns'):
               df = concatcolumns(df,tables['concatcolumns'][0],tables['concatcolumns'][1])
            if True:
               df = addauditcolumns(df)
            if True:
               if spark.catalog.tableExists(f"silver.da_{modulename}_dev.{table}"):
                  mergetotarget(df,tables['keycolumnsformerge'],f"silver.da_{modulename}_dev.{table}")
               else:
                  df.write.format("delta").mode("overwrite").saveAsTable(f"silver.da_{modulename}_dev.{table}")
   except Exception as e:
            error = str(e)
            print((f"Error in silver table {table} with {error}"))
            continue



